In [1]:
import pandas as pd
import re
from mlxtend.frequent_patterns import apriori, association_rules
data = pd.read_csv("Team_Sports_Survey_Preprocessed.csv")
# remove the eg. for each items
def remove_parentheses(text):
    if isinstance(text, str):  
        return re.sub(r"\s*\(.*?\)", "", text)
    return text  

In [2]:
# Clean the columns data by removing the (eg. items) by using the regex
columns_to_combine = ["Preferred Swimming Pool Activities", "Preferred Beach Activities", "Preferred Spa and Wellness Activities"]
data[columns_to_combine] = data[columns_to_combine].applymap(remove_parentheses)

# Combine all the columns
data["Combined Activities"] = data[columns_to_combine].apply(lambda x: ', '.join(x.dropna()), axis=1)

# Convert the items into a transactional data 
transactions = data["Combined Activities"].str.split(",").explode().str.strip().reset_index()
transactions_encoded = transactions.pivot_table(index="index", columns="Combined Activities", aggfunc=lambda x: 1, fill_value=0)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_27384\2987742297.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[columns_to_combine] = data[columns_to_combine].applymap(remove_parentheses)


In [3]:
# Use Apriori algorithm to find frequent association
frequent_items= apriori(transactions_encoded, min_support=0.2, use_colnames=True)# threshold 20%
rule = association_rules(frequent_items)
rule[['antecedents','consequents','support', 'confidence', 'lift']].sort_values("confidence", ascending=False)

C:\Users\joaqu\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift
25,"(Beach Swimming, Spa Treatment)",(Relaxing in Pool),0.31,0.968750,1.241987
19,"(Play with Pool Toys, Beach Swimming)",(Relaxing in Pool),0.28,0.965517,1.237843
71,"(Massage Therapy, Beach Swimming, Spa Treatment)",(Relaxing in Pool),0.27,0.964286,1.236264
39,"(Jetskiing, Spa Treatment)",(Relaxing in Pool),0.25,0.961538,1.232742
78,"(Sauna and Steam Rooms, Beach Swimming, Spa Tr...",(Relaxing in Pool),0.23,0.958333,1.228632
...,...,...,...,...,...
30,"(Kayaking/Paddling, Spa Treatment)",(Jetskiing),0.21,0.807692,1.969981
21,(Play with Pool Toys),"(Relaxing in Pool, Beach Swimming)",0.28,0.800000,1.379310
82,"(Jetskiing, Relaxing in Pool, Spa Treatment)",(Kayaking/Paddling),0.20,0.800000,1.632653
49,"(Practice Actual Swimming Techniques, Massage ...",(Spa Treatment),0.20,0.800000,1.904762
